**Import Libraries**

In [1]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import transforms as transforms
import numpy as np
import os
import argparse
import utils
from fer import FER2013
from torch.autograd import Variable
from models import *

**Parse Arguments**

In [2]:
class Opt:
    def __init__(self):
        self.model = 'ResNet18'
        self.dataset = 'FER2013'
        self.bs = 128
        self.lr = 0.01
        self.resume = True

opt = Opt()

**Setup and Configuration**

In [3]:
use_cuda = torch.cuda.is_available()
best_Testing_acc = 0 
best_Testing_acc_epoch = 0
best_Validation_acc = 0 
best_Validation_acc_epoch = 0
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

learning_rate_decay_start = 80  # 50
learning_rate_decay_every = 5   # 5
learning_rate_decay_rate = 0.9  # 0.9

cut_size = 44
total_epoch = 100

path = os.path.join(opt.dataset + '_' + opt.model)

#lists to save metrices
train_acc_list = []
train_loss_list = []
testing_acc_list = []
testing_loss_list = []
validation_acc_list = []
validation_loss_list = []

**Data preparation**

In [4]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(44),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

trainset = FER2013(split='Training', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=opt.bs, shuffle=True, num_workers=0)
Testingset = FER2013(split='Testing', transform=transform_test)
Testingloader = torch.utils.data.DataLoader(Testingset, batch_size=opt.bs, shuffle=False, num_workers=0)
Validationset = FER2013(split='Validation', transform=transform_test)
Validationloader = torch.utils.data.DataLoader(Validationset, batch_size=opt.bs, shuffle=False, num_workers=0)

==> Preparing data..


**Model setup**

In [5]:
if opt.model == 'ResNet18':
    net = ResNet18()
elif opt.model == 'VGG19':
    net = VGG('VGG19')

if not os.path.isdir(path):
    print("No checkpoint directory found. Starting training from scratch.")
    opt.resume = False

if opt.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    checkpoint = torch.load(os.path.join(path, 'Validation_model.t7'), map_location=torch.device('cpu'))
    net.load_state_dict(checkpoint['net'])
    best_Testing_acc = checkpoint['best_Testing_acc']
    best_Validation_acc = checkpoint['best_Validation_acc']
    best_Testing_acc_epoch = checkpoint['best_Testing_acc_epoch']
    best_Validation_acc_epoch = checkpoint['best_Validation_acc_epoch']
    start_epoch = checkpoint['best_Validation_acc_epoch'] + 1
else:
    print('==> Building model..')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=opt.lr, momentum=0.9, weight_decay=5e-4)

==> Resuming from checkpoint..


C:\Users\USER\AppData\Local\Temp\ipykernel_1984\218752443.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(path, 'Validation_model.t

**Traning Function**

In [6]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    global Train_acc
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate ** frac
        current_lr = opt.lr * decay_factor
        utils.set_lr(optimizer, current_lr)  # set the decayed rate
    else:
        current_lr = opt.lr
    print('learning_rate: %s' % str(current_lr))

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs, targets
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        utils.clip_gradient(optimizer, 0.1)
        optimizer.step()
        train_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).sum().item()

        utils.progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                           % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    Train_acc = 100.*correct/total
    train_acc_list.append(Train_acc)
    train_loss_list.append(train_loss / len(trainloader))

**Testing Function**

In [7]:
def Testing(epoch):
    global Testing_acc
    global best_Testing_acc
    global best_Testing_acc_epoch
    net.eval()
    Testing_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(Testingloader):
        bs, ncrops, c, h, w = np.shape(inputs)
        inputs = inputs.view(-1, c, h, w)
        inputs, targets = inputs, targets
        with torch.no_grad():
            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
            loss = criterion(outputs_avg, targets)
            Testing_loss += loss.item()
            _, predicted = torch.max(outputs_avg.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).sum().item()

        utils.progress_bar(batch_idx, len(Testingloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                           % (Testing_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    # Save checkpoint.
    Testing_acc = 100.*correct/total
    if Testing_acc > best_Testing_acc:
        print('Saving..')
        print("best_Testing_acc: %0.3f" % Testing_acc)
        state = {
            'net': net.state_dict(),
            'acc': Testing_acc,
            'epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path, 'Testing_model.t7'))
        best_Testing_acc = Testing_acc
        best_Testing_acc_epoch = epoch

    testing_acc_list.append(Testing_acc)
    testing_loss_list.append(Testing_loss / len(Testingloader))

**Validation Function**

In [8]:
def Validation(epoch):
    global Validation_acc
    global best_Validation_acc
    global best_Validation_acc_epoch
    net.eval()
    Validation_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for batch_idx, (inputs, targets) in enumerate(Validationloader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)
            inputs, targets = inputs, targets

            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
            loss = criterion(outputs_avg, targets)
            Validation_loss += loss.item()

            _, predicted = torch.max(outputs_avg.data, 1)
            correct += predicted.eq(targets.data).sum().item()
            total += targets.size(0)  # Update the total number of samples

            utils.progress_bar(batch_idx, len(Validationloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                               % (Validation_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    # Save checkpoint if this is the best accuracy
    Validation_acc = 100. * correct / total
    if Validation_acc > best_Validation_acc:
        print('Saving..')
        print("best_Validation_acc: %0.3f" % Validation_acc)
        state = {
            'net': net.state_dict(),
            'best_Testing_acc': best_Testing_acc,
            'best_Validation_acc': Validation_acc,
            'best_Testing_acc_epoch': best_Testing_acc_epoch,
            'best_Validation_acc_epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path, 'Validation_model.t7'))
        best_Validation_acc = Validation_acc
        best_Validation_acc_epoch = epoch

    validation_acc_list.append(Validation_acc)
    validation_loss_list.append(Validation_loss / len(Validationloader))

**Traning Loop**

In [9]:
for epoch in range(start_epoch, total_epoch):
    train(epoch)
    Testing(epoch)
    Validation(epoch)


Epoch: 77
learning_rate: 0.01


F:\FER-2013 Project\fer2013_vgg_resnet\transforms\functional.py:63: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


 [=============================>] | Loss: 0.151 | Acc: 94.646% (27172/28709)    225/225 
 [============================>.] | Loss: 1.230 | Acc: 69.936% (2510/3589)      29/29 
 [============================>.] | Loss: 1.230 | Acc: 69.936% (2510/3589)      29/29 

Epoch: 78
learning_rate: 0.01
 [=============================>] | Loss: 0.157 | Acc: 94.552% (27145/28709)    225/225 
 [============================>.] | Loss: 1.358 | Acc: 67.623% (2427/3589)      29/29 
 [============================>.] | Loss: 1.358 | Acc: 67.623% (2427/3589)      29/29 

Epoch: 79
learning_rate: 0.01
 [=============================>] | Loss: 0.159 | Acc: 94.389% (27098/28709)    225/225 
 [============================>.] | Loss: 1.193 | Acc: 69.908% (2509/3589)      29/29 
 [============================>.] | Loss: 1.193 | Acc: 69.908% (2509/3589)      29/29 

Epoch: 80
learning_rate: 0.01
 [=============================>] | Loss: 0.153 | Acc: 94.646% (27172/28709)    225/225 
 [===========================

In [10]:
print("best_Testing_acc: %0.3f" % best_Testing_acc)
print("best_Testing_acc_epoch: %d" % best_Testing_acc_epoch)
print("best_Validation_acc: %0.3f" % best_Validation_acc)
print("best_Validation_acc_epoch: %d" % best_Validation_acc_epoch)

best_Testing_acc: 71.719
best_Testing_acc_epoch: 94
best_Validation_acc: 71.719
best_Validation_acc_epoch: 94


In [12]:
import pickle

metrics = {
    'train_acc_list': train_acc_list,
    'train_loss_list': train_loss_list,
    'testing_acc_list': testing_acc_list,
    'testing_loss_list': testing_loss_list,
    'validation_acc_list': validation_acc_list,
    'validation_loss_list': validation_loss_list
}

with open(os.path.join(path, 'metrics.pkl'), 'wb') as f:
    pickle.dump(metrics, f)